In [74]:
import tweepy
import plotly
import datetime
import pytz

import matplotlib as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from datetime import timedelta

In [75]:
# Keys
access_token = '1258532548134797313-w24ae6Fc7uIHO0jXk40akPLrFfUvMW'

access_token_secret = 'L8PwULD3RHMYHnxc7cJlvZSYtXTQORc0ElEFIfL1S7V21'

consumer_key = 'YuJdN38YPbdwFJpAUooTHhGbh'

consumer_key_secret = 'IOzsqk0Vly9Y90xhXqXQqQ1tsyLS11zgVTjGnpdMTkYJzKH2Dl'

# Start the API 
auth = tweepy.OAuthHandler(consumer_key,consumer_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [76]:
tweets_list = []

for t in tweepy.Cursor(api.user_timeline, id = 'GGallegos24').items():
    tweet_id = t.id
    try:
        # Create dictionary for specific follower and save data in there
        tweet = {}
        tweet['id'] = t.id
        tweet['text'] = t.text

        tweet['hashtags'] = t.entities['hashtags']
        tweet['user_mentions'] = t.entities['user_mentions']
        
        tweet['source'] = t.source

        tweet['created_at'] = t.created_at    

    except:
        pass
    
    tweets_list.append(tweet)

In [84]:
# Create Panda DF for local use
df = pd.DataFrame(tweets_list)

In [136]:
# Create Custom Columns

## RT
def rt_count(text):
    if text[0:2] == 'RT':
        return('RT')
    else:
        return('Original Tweet')
df['RT_Status'] = df['text'].apply(rt_count)

## Parse Source
def parse_source(source):
    index = (len(source)-7)
    return(source[index:])

df['source'] = df['source'].apply(parse_source)

## Parse user_mention
mentions = list(df['user_mentions'])

for i in range(len(mentions)):
    try:
        df.loc[i,'user_mention'] = mentions[i][0]['screen_name']
        df.loc[i,'user_mention_name'] = mentions[i][0]['name']
    except:
        df.loc[i,'user_mention'] = ''
        df.loc[i,'user_mention_name'] = ''

## Parse Hashtags
hashtags = list(df['hashtags'])

for i in range(len(mentions)):
    try:
        df.loc[i,'hashtag'] = hashtags[i][0]['text']
    except:
        df.loc[i,'hashtag'] = ' '

## Change timezone in created_at
def change_tz(created_at):
    utc_moment = created_at.replace(tzinfo = pytz.utc)
    return(utc_moment.astimezone(pytz.timezone('America/El_Salvador')))

df['created_at'] = df['created_at'].apply(change_tz)

## Parse dates
def parse_created_at(created_at):
    return(datetime.datetime.strftime(created_at,'%Y-%m-%d'))
def parse_weekday(created_at):
    return(created_at.strftime('%A'))
def parse_date(created_at):
    return(created_at.day)
def parse_month(created_at):
    return(created_at.strftime('%B'))
def parse_year(created_at):
    return(created_at.year)

## Parse hours
def parse_hour(created_at):
    return(created_at.hour)

df['hour'] = df['created_at'].apply(parse_hour)
df['date'] = df['created_at'].apply(parse_date)
df['weekday'] = df['created_at'].apply(parse_weekday)
df['month'] = df['created_at'].apply(parse_month)
df['year'] = df['created_at'].apply(parse_year)
df['created_at_date'] = df['created_at'].apply(parse_created_at)

## Add column to sort weekdays by numbers
def sort_weekday(created_at):
    n= created_at.weekday()
    return((-1*n) + 7)

df['sort_weekday'] = df['created_at'].apply(sort_weekday)

## Add column to sort months
def sort_month(created_at):
    return(created_at.month)

df['sort_month'] = df['created_at'].apply(sort_month)

## Tweet length
df['tweet_length'] = df['text'].apply(len)

print(df.head(5))

id                                               text  \
0  1274755897223512064  RT @SolisEduardo_: Muchas personas confundiero...   
1  1274755482222239746  RT @elblog: #Nacionales_EB\n\nEstados Unidos d...   
2  1274753871630479363  RT @NoticieroHechos: La noticia más difícil qu...   
3  1274751030501269505  RT @GracielaRajo: Nuestro corazón esta de luto...   
4  1274750939015139331  RT @CrodriguezSV: San Salvador https://t.co/CT...   

                                           hashtags  \
0                                                []   
1  [{'text': 'Nacionales_EB', 'indices': [12, 26]}]   
2                                                []   
3                                                []   
4                                                []   

                                       user_mentions   source  \
0  [{'screen_name': 'SolisEduardo_', 'name': 'Edu...  Android   
1  [{'screen_name': 'elblog', 'name': 'El Blog', ...  Android   
2  [{'screen_name': 'NoticieroH

In [137]:
# PLOTS
## Most common hashtags
df_2 = df.groupby('hashtag').count().reset_index()
### Drop the first value because that is the value of empty string
df_2.drop(0, inplace=True)
### Order values by count
df_2 = df_2.sort_values(by = 'hashtags', ascending = False)
### Select top 10
df_2 = df_2.head(10)
fig = px.bar(
    df_2,
    y = 'hashtag',
    x = 'text',
    orientation='h',
    title = 'Hashtag: Top 10',
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()


In [138]:
## Most common mentioned users
df_2 = df.groupby('user_mention_name').count().reset_index()
### Drop the first value because that is the value of empty string
df_2.drop(0, inplace=True)
### Order values by count
df_2 = df_2.sort_values(by = 'text', ascending = False)
### Select top 10
df_2 = df_2.head(10)
fig = px.bar(
    df_2,
    y = 'user_mention_name',
    x = 'text',
    orientation='h',
    title = 'Mentioned Users: Top 10',
)
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()

In [139]:
## Percent Original Tweets vs Retweets
## Most common mentioned users
df_2 = df.groupby('RT_Status').count().reset_index()
### Order values by count
df_2 = df_2.sort_values(by = 'text', ascending = False)

fig = px.pie(
    df_2,
    values = 'text',
    names = 'RT_Status',
    title = 'Percent Retweets vs Original Tweets',
)
fig.show()

In [140]:
df_3 = df.groupby(['weekday','hour','sort_weekday']).count().reset_index()
df_3 = df_3.sort_values(by = 'sort_weekday')
fig = px.scatter(
            df_3,
            y = 'weekday', 
            x = 'hour',
            color = 'text', 
            size = 'text')
fig.update_layout(title = 'Numero de Tuits: Hora y Dia', xaxis_title='Hora', yaxis_title='Dia de la Semana', legend_title_text = 'Numero de Tuits')

fig.show()

In [217]:
df_4 = df.groupby(['weekday','hour','sort_weekday','tweet_length']).mean().reset_index()

df_4 = df_4.sort_values(by = 'sort_weekday')
fig = px.scatter(
            df_4,
            y = 'weekday', 
            x = 'hour',
            color = 'tweet_length', 
            size = 'tweet_length')
fig.update_layout(title = 'Numero de Caracteres Promedio por Tuit: Hora y Dia', xaxis_title='Hora', yaxis_title='Dia de la Semana')

fig.show()

In [196]:
fig = go.Figure()

## Subset for totals
df_4 = df.groupby(['created_at_date','month']).count().reset_index()
df_4 = df_4.sort_values(by='created_at_date')

## Subset for only RT
df_4_RT = df.loc[df['RT_Status'] == 'RT']

df_4_RT = df_4_RT.groupby(['created_at_date','month']).count().reset_index()

df_4_RT = df_4_RT.sort_values(by='created_at_date')

fig.add_trace(go.Bar(x = df_4_RT['created_at_date'],y = df_4_RT['text'],name = 'Retuits'))

## Subset for only RT
df_4_OT = df.loc[df['RT_Status'] == 'Original Tweet']

df_4_OT = df_4_OT.groupby(['created_at_date','month']).count().reset_index()

df_4_OT = df_4_OT.sort_values(by='created_at_date')

fig.add_trace(go.Bar(x = df_4_OT['created_at_date'],y = df_4_OT['text'],name = 'Tuits originales',text = df_4['text'],textposition='outside', textfont_size = 20))

## Update layout
fig.update_layout(title = 'Numero de Tuits por Dia:RT, y OT',barmode='stack',width = 1000)
fig.show()

In [207]:
df_5 = df.groupby('hour').mean().reset_index() 
fig = px.bar(
    df_5,
    x = 'hour',
    y = 'tweet_length'
    )


fig.show()